In [1]:
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import numpy as np
import numpy as np
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
import torch
import tensorflow
import numpy as np
import matplotlib as mpl

## load finite net data

In [2]:
x = np.load('invBurg_train.npy')[:]
mean = np.mean(x)
std = np.std(x)
x = x.reshape(2000,101,10,10,1)
x = (x-mean)/std


s,f,r,w,c = x.shape
z = np.zeros((s,f,r+2,w+2,c))
z[:,:,1:-1,1:-1,:]  = x

In [12]:
mpl.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})

mpl.rcParams['figure.dpi'] = 300
mpl.rcParams.update({'font.size': 25})
mpl.rcParams['axes.linewidth'] = 1
from matplotlib import ticker
formatter = ticker.ScalarFormatter(useMathText=True)
formatter.set_scientific(True) 


a = np.load('invBurg_train.npy')[:]

plt.figure(figsize=(10,10));plt.tight_layout()
plt.subplot(1,2,1)
plt.imshow(a[1]);plt.xlabel('$x$');plt.ylabel('$t$')
plt.subplot(1,2,2);plt.imshow(a[50]);plt.xlabel('$x$');plt.ylabel('$t$')
plt.savefig('burger.svg')

ModuleNotFoundError: No module named 'torch'

## Repeat the data

In [4]:
xr = np.copy(z)
for  i in range(6) : xr= np.concatenate([xr,xr])
    
xr.shape

(128000, 101, 12, 12, 1)

In [24]:
def generate_data_batches(input_data,
                          steps=1,
                          seed=42,
                          max_iters = 100,
                          batch_size=32,
                          batches=100,
                          progress=True,
                          step_size = 10,
                          key_bias=0,
                          save_file = None):
    '''
    return dictionary with key of the batch number
    '''
    
    np.random.seed(seed)
    iters_list=  np.random.randint(low=step_size,high=max_iters-step_size*(steps-1)+1,size=batches)

    #scaling 
    mean = 0 #(tR[1]-tR[0])/2
    std  = 1 #mean/2
    data={}
    
    for batch_num in tqdm(range(batches)):
        
        iter_n = iters_list[batch_num]
        extract_index = np.arange(iter_n-step_size,iter_n+step_size*steps,step_size)
        generated_data = input_data[batch_num:batch_num+batch_size,extract_index,:,:,:]
#         print(batch_num,batch_num+batch_size)
#         print(generated_data)
#         break
        if save_file is None :
            data[f'{batch_num + key_bias}'] = (generated_data) 
        else : 
            hf.create_dataset(f'{batch_num}',data = data[batch_num] , compression ='gzip')
    
    if save_file is not None :hf.close()
    else : return data

    
def data_generator(data):
    '''
    input : data dictionary (batch number :5D tensor data)
    output: input , target values
    '''
    batches = len(data.keys())
    batch_size = len(data['0'])
    batch_counter= 0
    
    while True:
        x,y = data[f'{batch_counter}'][:,:-1,:,:,:],data[f'{batch_counter}'][:,-1:,:,:,:]

        batch_counter += 1
        yield x,y
        if batch_counter == batches:batch_counter = 0

In [25]:
train_data_batches =generate_data_batches(xr,
                                          steps=1,
                                          max_iters = 100,
                                          seed=42,
                                          batch_size=32,
                                          step_size=10,
                                          batches=xr.shape[0]//32,
                                          progress=True)
#                                           save_file = 'N=100x100 batch_size=64 batches=5000 max_iters=1000')

In [28]:
## from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import *
from tensorflow.keras.layers import *
import tensorflow
# def dense_block(c0,f,k,s,f1,f2,f3,k1,k2,k3):
#     i0 = TimeDistributed(Conv2D(f1,k1,strides=s,padding='same',kernel_initializer='glorot_uniform',activation=LeakyReLU()))(c0)
#     i0 = Concatenate()([c0,i0])
#     j0 = TimeDistributed(Conv2D(f2,k2,strides=s,padding='same',kernel_initializer='glorot_uniform',activation=LeakyReLU()))(i0)
#     j0 =Concatenate()([j0,i0])
#     k0 = TimeDistributed(Conv2D(f3,k3,strides=s,padding='same',kernel_initializer='glorot_uniform',activation=LeakyReLU()))(j0)
#     k0 =Concatenate()([j0,k0])
#     return k0


def conv(x,f,k):
    x = TimeDistributed(Conv2D(f,(k,k),strides=1,padding='same',kernel_initializer='glorot_uniform',activation=LeakyReLU()))(x)
    return x
    
def deconv(x,f,k):
    x = TimeDistributed(Conv2DTranspose(f,(k,k),strides=1,padding='same',kernel_initializer='glorot_uniform',activation=LeakyReLU()))(x)
    return x
    
    
def dense_block(tensor, f, r,k):
    for _ in range(r):
        x = conv(tensor, f=4*f, k=1)
        x = conv(x, f=f, k=k)
        tensor = Concatenate()([tensor, x])
    return tensor

def inv_dense_block(tensor, f, r,k):
    for _ in range(r):
        x = deconv(tensor, f=4*f, k=1)
        x = deconv(x, f=f, k=k)
        tensor = Concatenate()([tensor, x])
    return tensor

def transition(x,s):
    ff = int(tensorflow.keras.backend.int_shape(x)[-1] * 0.5)
    m0 = TimeDistributed(Conv2D(ff,(1,1),strides=2*s,padding='same',kernel_initializer='glorot_uniform',activation=LeakyReLU()))(x)
    return m0

def inv_transition(x,s):
    ff = int(tensorflow.keras.backend.int_shape(x)[-1] * 0.5)
    m0 = TimeDistributed(Conv2DTranspose(ff,(1,1),strides=2*s,padding='same',kernel_initializer='glorot_uniform',activation=LeakyReLU()))(x)
    return m0


def dfn():
    
    k=3#hp.Choice('k',values=[2,3])
    
    s=1;LR=1e-3
    
    
    r1 = 2#hp.Choice('r1',values=[1,2,3,4,5,6,7,8])
    r2 = 4#hp.Choice('r2',values=[1,2,3,4,5,6,7,8])
    r3 = 8#hp.Choice('r3',values=[1,2,3,4,5,6,7,8])
    
    f0 = 128#hp.Choice('f0',values=[4,8,16,32,64,128,256]) 
    f1 = 32#hp.Choice('fd',values=[4,8,16,32,64])
    f2 = 32#hp.Choice('f2',values=[4,8,16,32,64])
    f3 = 32#hp.Choice('f3',values=[4,8,16,32,64]) 
    
    l1 = 128#hp.Choice('l1',values=[4,8,12,16,24,32,64,96,128,256])
    l2 = 64#hp.Choice('l2',values=[4,8,12,16,24,32,64,96,128,256])
    
    
    x = Input(shape=(None, 12,12, 1))
    c0 = TimeDistributed(Conv2D(f0,(k,k),strides=1,padding='same',kernel_initializer='glorot_uniform',activation=LeakyReLU()))(x)
########################################################################################################    
    e1 = dense_block(c0,f1,r=r1,k=k);m1 = transition(e1,s)
    e2 = dense_block(m1,f2,r=r2,k=k);m2 = transition(e2,s)
    e3 = dense_block(m2,f3,r=r3,k=k);
########################################################################################################
    e = ConvLSTM2D(l1,(2,2),padding='same',return_sequences=True)(e3)
    b = ConvLSTM2D(l2,(2,2),padding='same',return_sequences=True)(e)
    d = ConvLSTM2D(l1,(2,2),padding='same',return_sequences=True)(b)
########################################################################################################
    d1 = inv_dense_block(d ,f3,r=r3,k=k);m1 = inv_transition(d1,s)
    d2 = inv_dense_block(m1,f2,r=r2,k=k);m2 = inv_transition(d2,s)
    d3 = inv_dense_block(m2,f1,r=r1,k=k);
########################################################################################################
    out = conv(d3,f=1,k=1)
    model = Model(x,out)
    optimizer = Adam(learning_rate=LR)
    model.compile(loss='mae',optimizer=optimizer,metrics=['mse'])
    
    params = model.count_params()/1e6
    print(params)
#     if params > 4 : return None
    return model

model = dfn()
# model.summary()



# keras.utils.plot_model(model,show_shapes=True)

3.567761


In [7]:
import tensorflow.keras as keras
history={}
i = 100

In [32]:
### 100 step

i+=1

 
early_stopping = keras.callbacks.EarlyStopping(
    monitor='loss',
    min_delta=5e-5, 
    patience=25, 
    verbose=1, 
    mode='auto',
    baseline=None, 
    restore_best_weights=False
)

csv_logger = tensorflow.keras.callbacks.CSVLogger('finiteNet - 3 r=1,2,3 f=256,32,32,32.log')
reduce_lr_callback = tensorflow.keras.callbacks.ReduceLROnPlateau(monitor = 'loss', 
                                      factor = 0.5, 
                                      patience = 3, 
                                      verbose = 1, 
                                      cooldown=1,
                                      min_delta = 1e-4,
                                      min_lr=1e-8 )

model.fit(       data_generator(train_data_batches),
                    steps_per_epoch=len(train_data_batches),
                    verbose=1,
                    epochs=1,
                    callbacks=[reduce_lr_callback,csv_logger,early_stopping],

                   )

4000/4000 [==============================] - 294s 74ms/step - loss: 0.0019 - mse: 1.1062e-04


In [35]:
model.save('burger std=0.7968797813840656 mean=1.4804453986909523 loss=0.0019.h5')

In [34]:
std,mean

(0.7968797813840656, 1.4804453986909523)

In [3]:
import sklearn.metrics
import sklearn

In [20]:
mean , std

(1.479030106840812, 0.7964541244780471)